<span style="float:right;"><a href="https://opensource.org/licenses/MIT">
    <img src="https://img.shields.io/badge/License-MIT-green.svg" />
</a></span>
<span style="float:right; display:inline-block; margin: 2.5px 5px;"><a href="https://creativecommons.org/licenses/by/4.0/">
    <img src="https://licensebuttons.net/l/by/4.0/80x15.png" />
</a></span>

# pyBarSim in 2.5D

**Author:** Guillaume Rongier
<br></br>

In this notebook, we will look at simulating the deposits of wave-dominated shallow-marine environments in 2.5D using pyBarSim.

### Imports

In [ ]:
import numpy as np
import pyvista as pv

from pybarsim import BarSimPseudo3D

## 1. Setup and run

Define the initial elevation and cell size (in m):

In [ ]:
initial_elevation = np.linspace(np.linspace(1000., 900., 200), np.linspace(1000., 850., 200), 150)

spacing = (100., 100.)

Define the run time (in yr) and the inflection points for the variations of sea level (in m):

In [ ]:
run_time = 25000.

sea_level_curve = np.array([
    (0., 998.),
    (0.25*run_time, 985.),
    (0.5*run_time, 975.),
    (0.75*run_time, 985.),
    (run_time + 20., 998.)
])

Define the inflection points for the sediment supply, which varies along-shore (in m$^2$/yr):

In [ ]:
sediment_supply_curve = np.array([
    np.tile([[0.], [0.25*run_time], [0.5*run_time], [0.75*run_time], [run_time + 20.]], 200),
    np.vstack([
        np.linspace(25., 5., 200),
        np.linspace(25., 5., 200),
        np.linspace(25., 5., 200),
        np.linspace(5., 1., 200),
        np.linspace(5., 1., 200),
    ])
]).T

Initialize a `BarSimPseudo3D` object and run the simulation:

<div class="alert alert-block alert-warning">
<b>&#9888;</b> This takes more time to run the first time because Numba needs to compile the Python code.
</div>

In [ ]:
barsim = BarSimPseudo3D(initial_elevation,
                        sea_level_curve,
                        sediment_supply_curve,
                        spacing=spacing,
                        max_wave_height_fair_weather=1.5,
                        allow_storms=True,
                        start_with_storm=False,
                        max_wave_height_storm=6.,
                        tidal_amplitude=2.,
                        min_tidal_area_for_transport=100.,
                        sediment_size=(5., 50., 125., 250.),
                        sediment_fraction=(0.25, 0.25, 0.25, 0.25),
                        initial_substratum=(200., (0.25, 0.25, 0.25, 0.25)),
                        erodibility=0.1,
                        washover_fraction=0.5,
                        tide_sand_fraction=0.3,
                        depth_factor_backbarrier=5.,
                        depth_factor_shoreface=10.,
                        local_factor_shoreface=1.5,
                        local_factor_backbarrier=1.,
                        fallout_rate_backbarrier=0.,
                        fallout_rate_shoreface=0.0002,
                        max_width_backbarrier=500.,
                        curve_preinterpolation=None,
                        seed=42)
barsim.run(850., 1000., 0.5, run_time, dt_fair_weather=15., dt_storm=1.)

## 2. Stratigraphy visualization

Here `run` directly creates `record_`, so a 3D structured regular grid:

<div class="alert alert-block alert-warning">
<b>&#9888;</b> <code>sequence_</code> is not implemented in <code>BarSimPseudo3D</code>.
</div>

In [ ]:
barsim.record_

And `finalize` is directly applied to `record_` to compute the mean grain size, the sorting term, and the major facies:

In [ ]:
barsim.finalize()

In [ ]:
barsim.record_

We can visualize sections through the resulting 3D grid using [xarray's plotting functions](https://docs.xarray.dev/en/stable/user-guide/plotting.html?highlight=plotting):

In [ ]:
barsim.record_['Mean grain size'][-10].plot(figsize=(8, 6))

In [ ]:
barsim.record_['Mean grain size'][:, 100].plot(figsize=(12, 5))

In [ ]:
barsim.record_['Deposits'][0, :, 100].plot(figsize=(12, 5))

## 3. 3D visualization

We first need to create a [PyVista](https://docs.pyvista.org/) mesh for 3D visualization of the sedimentary deposits:

In [ ]:
mesh = barsim.mesh(zscale=50.)

We can then visualize the full 2.5D deposits:

In [ ]:
p = pv.Plotter()
p.add_mesh(mesh.threshold(), scalars='Mean grain size', lighting=False)
p.show()

And visualize slices through the deposits:

In [ ]:
p = pv.Plotter()
p.add_mesh(mesh.threshold().slice_along_axis(n=10, axis='x'), scalars='Major facies', lighting=False)
p.add_mesh(mesh.threshold().slice_along_axis(n=10, axis='y'), scalars='Major facies', lighting=False)
p.show()